In [ ]:
import pandas as pd
import numpy as np

df_kl=pd.read_excel('~/Downloads/kl-1.xlsx')
df_kl.insert(loc=0,column='序号',value = [1,5,6,2,4,3])
df_kl_t=df_kl.sort_values(by=['序号'],ascending=True)
page_2 = df_kl_t[['项目','日均客流','同期日均客流','日均客流同比','平日日均客流','周末日均客流','周末/平日比','同期周末/平日比']]
page_2.astype({'周末/平日比':'float','同期周末/平日比':'float'})
page_2['周末占比同比'] = df_kl_t['周末/平日比'] - df_kl_t['同期周末/平日比']
page_2 = df_kl_t[['项目','日均客流','同期日均客流','日均客流同比','平日日均客流','周末日均客流','周末/平日比','周末占比同比']]

df_xs=pd.read_excel('~/Downloads/xs-1.xlsx')
df_all = df_xs.loc[(df_xs.项目分类 =='项目整体')]
df_all.insert(loc=0,column='序号',value =[1,5,6,2,4,3])
df_xs_a=df_all.sort_values(by=['序号'],ascending=True)
page_3 = df_xs_a[['项目','日均销售额','同期日均销售额','日均销额售同比','平日日均销售额','周末日均销售额','周末/平日比','周末占比同比']]


df_major = df_xs.loc[(df_xs.项目分类 =='主次力店')]
df_major.insert(loc=0,column='序号',value =[1,5,6,2,4,3])
df_xs_m=df_major.sort_values(by=['序号'],ascending=True)
page_4 = df_xs_m[['项目','日均销售额','同期日均销售额','日均销额售同比','平日日均销售额','周末日均销售额','周末/平日比','周末占比同比']]


df_normal = df_xs.loc[(df_xs.项目分类 =='一般店铺')]
df_normal.insert(loc=0,column='序号',value =[1,5,6,2,4,3])
df_xs_n=df_normal.sort_values(by=['序号'],ascending=True)
page_5 = df_xs_n[['项目','日均销售额','同期日均销售额','日均销额售同比','平日日均销售额','周末日均销售额','周末/平日比','周末占比同比']]



page_1 = pd.merge(page_2,page_3,on='项目',how='left')[['项目','日均客流','日均客流同比','日均销售额','日均销额售同比']]


rjkl=page_2.日均客流.sum()/6
rjkl_tq = page_2.同期日均客流.sum()
rjkl_tb = (rjkl-rjkl_tq)/rjkl_tq

rjxs=page_3.日均销售额.sum()/6
rjxs_tq=page_3.同期日均销售额.sum()
rjxs_tb=(rjxs-rjxs_tq)/rjxs_tq

df_a_row1 = pd.DataFrame(np.insert(page_1.values,0,values=['均值',rjkl_tq,rjkl_tb,rjxs_tq,rjxs_tb],axis=0))

page_2=page_2.round(decimals=1)
page_3=page_3.round(decimals=0)
page_4=page_4.round(decimals=0)
page_5=page_5.round(decimals=0)
page_1=page_1.round(decimals=0)

with pd.ExcelWriter('~/Downloads/ppt.xlsx') as writer:
    df_a_row1.to_excel(writer,sheet_name='page_1')
    page_2.to_excel(writer,sheet_name='page_2')
    page_3.to_excel(writer,sheet_name='page_3')
    page_4.to_excel(writer,sheet_name='page_4')
    page_5.to_excel(writer,sheet_name='page_5')
    


In [ ]:
import pandas as pd
import numpy as np

df_1 = pd.read_excel('~/Downloads/南京弘阳广场客流202105.xls',header=5)
df_ps= df_1.loc[(df_1.属性 =='平时')][['日期','星期','属性','P00011']].sort_values(by='日期').rename(columns={'P00011':'客流'})
df_zm= df_1.loc[(df_1.属性 =='周末')][['日期','星期','属性','P00011']].sort_values(by='日期').rename(columns={'P00011':'客流'})

df_ps=df_ps.groupby(df_ps.属性).agg({'客流':'sum'}).reset_index()
df_zm=df_zm.groupby(df_zm.属性).agg({'客流':'sum'}).reset_index()
df_ps 

In [ ]:
import pandas as pd
import numpy as np
from output_style_format import to_excel_auto_column_weight

def get_df(folderpath,shoppingmall,month:str):
    #folderpath,month1,month2,shoppingmall,form
    df_xs_all = pd.read_excel(folderpath+shoppingmall+month+'.xlsx')
    df_ppk = pd.read_excel(folderpath+'/品牌库.xlsx')


    df_xs_all =df_xs_all[['项目名称','铺位类型','品牌名称','营业额','营业额日期']]
    df_ppk = df_ppk[['品牌名称','业态']]

    df_xs_all_ppk = pd.merge(df_xs_all,df_ppk,on='品牌名称',how='left')

    df_xs_zcld = df_xs_all_ppk.loc[(df_xs_all_ppk.业态=='主次力店')]
    df_xs_zcld = df_xs_zcld.groupby(df_xs_zcld.营业额日期).agg({'营业额':'sum'}).reset_index()

    df_xs_ybdp = df_xs_all_ppk.loc[(df_xs_all_ppk.业态!='主次力店')]
    df_xs_ybdp = df_xs_ybdp.groupby(df_xs_ybdp.营业额日期).agg({'营业额':'sum'}).reset_index()

    df_xs_zcld['营业额日期']=df_xs_zcld['营业额日期'].apply(lambda x : x.strftime('%Y-%m-%d'))
    df_xs_ybdp['营业额日期']=df_xs_ybdp['营业额日期'].apply(lambda x : x.strftime('%Y-%m-%d'))

    return df_xs_zcld,df_xs_ybdp

df1,df2=get_df('~/Desktop/销售客流pandas/','合肥弘阳广场','202105')
df5,df6=get_df('~/Desktop/销售客流pandas/','合肥弘阳广场','202205')

with pd.ExcelWriter('~/Downloads/xs_合肥弘阳广场1.xlsx',engine="openpyxl") as writer:
    to_excel_auto_column_weight(df1,writer,'主次力店202105')
    to_excel_auto_column_weight(df2,writer,'一般店铺202105')
    to_excel_auto_column_weight(df5,writer,'主次力店202205')
    to_excel_auto_column_weight(df6,writer,'一般店铺202205')



In [ ]:
import pandas as pd
import numpy as np

from datetime import datetime


def get_diff_xs(folderpath,month1,month2,shoppingmall,form):
    """
    Parameters: Str: month1,month2,shoppingmall
    get_diff_xs('~/Desktop/销售客流pandas/','202105','202205','合肥弘阳广场',0)
    form:0-所有业态；1-主次力店；2-一般商铺
    Return :
    项目名称	 平日销售日均	 周末销售日均	周末/平日比	周末占比同比
    合肥弘阳广场	788503.48	1123773.68  	0.58	-0.39
    """
    #导入数据并整理
    df_xs_zp = pd.read_excel(folderpath+shoppingmall+month1+'.xlsx')
    df_xs_dj = pd.read_excel(folderpath+shoppingmall+month2+'.xlsx')
    df_ppk = pd.read_excel('~/Desktop/销售客流pandas/品牌库.xlsx')
    df_xs_zp =df_xs_zp[['项目名称','铺位类型','品牌名称','营业额','营业额日期']]
    df_xs_dj =df_xs_dj[['项目名称','铺位类型','品牌名称','营业额','营业额日期']]
    df_xs_zp['营业额日期']=df_xs_zp.营业额日期.apply(lambda x:x.date())
    df_xs_dj['营业额日期']=df_xs_dj.营业额日期.apply(lambda x:x.date())

    df_ppk = df_ppk[['品牌名称','业态']]
    df_xs_all_ppk1 = pd.merge(df_xs_zp,df_ppk,on='品牌名称',how='left')
    df_xs_all_ppk2 = pd.merge(df_xs_dj,df_ppk,on='品牌名称',how='left')
    #根据业态，区分销售数据的统计范围
    if form==1:
        df_xs_all_ppk1=df_xs_all_ppk1.loc[(df_xs_all_ppk1.业态=='主次力店')]
        df_xs_all_ppk2=df_xs_all_ppk2.loc[(df_xs_all_ppk2.业态=='主次力店')]
    if form==2:
        df_xs_all_ppk1=df_xs_all_ppk1.loc[(df_xs_all_ppk1.业态!='主次力店')]
        df_xs_all_ppk2=df_xs_all_ppk2.loc[(df_xs_all_ppk2.业态!='主次力店')]

    #month1 周末平日销售结果
    df_xs_all_ppk1['周末'] = df_xs_all_ppk1['营业额日期'].apply(lambda x: '是' if x.weekday() >4 else '否')
    df_tmp1 = df_xs_all_ppk1.groupby(by='周末').agg({'营业额':'sum'}).reset_index()
    df_tmp1['日均营业额']=df_tmp1.apply(lambda x:x['营业额']/9 if x['周末']=='是' else x['营业额']/22,axis=1)

    a,b,c=df_tmp1.iat[0,2],df_tmp1.iat[1,2],df_tmp1.iat[1,1]/df_tmp1.iat[0,1]
    df_xs_res1 = pd.DataFrame(data={'项目名称':['合肥弘阳广场'],'平日销售日均':[a],'周末销售日均':[b],'周末/平日比':[c]})
    df1 = df_xs_res1.reset_index()

    #month2 周末平日销售结果
    df_xs_all_ppk2['周末'] = df_xs_all_ppk2['营业额日期'].apply(lambda x: '是' if x.weekday() >4 else '否')
    df_tmp2 = df_xs_all_ppk2.groupby(by='周末').agg({'营业额':'sum'}).reset_index()
    df_tmp2['日均营业额']=df_tmp2.apply(lambda x:x['营业额']/9 if x['周末']=='是' else x['营业额']/22,axis=1)

    a,b,c=df_tmp2.iat[0,2],df_tmp2.iat[1,2],df_tmp2.iat[1,1]/df_tmp2.iat[0,1]
    df_xs_res2 = pd.DataFrame(data={'项目名称':['合肥弘阳广场'],'平日销售日均':[a],'周末销售日均':[b],'周末/平日比':[c]})
    df2 = df_xs_res2.reset_index()

    #输出同比结果
    df2['上期周末占比']=df1.iat[0,4]
    df2['周末占比同比']=df2.iat[0,4]-df1.iat[0,4]

    return df2.round(2)

df_all = get_diff_xs('~/Desktop/销售客流pandas/','202105','202205','合肥弘阳广场',0)
df_zcld = get_diff_xs('~/Desktop/销售客流pandas/','202105','202205','合肥弘阳广场',1)
df_ybsp = get_diff_xs('~/Desktop/销售客流pandas/','202105','202205','合肥弘阳广场',2)

with pd.ExcelWriter('~/Downloads/xs_hefei.xlsx') as writer:
    df_all.to_excel(writer,sheet_name='所有业态')
    df_zcld.to_excel(writer,sheet_name='主次力店')
    df_ybsp.to_excel(writer,sheet_name='一般商铺')


In [ ]:
import pandas as pd
import numpy as np

df_tj =pd.read_excel('~/Downloads/xs_hf_2205.xlsx',sheet_name='统计表',header=2)
df_hf_202105 =pd.read_excel('~/Downloads/xs_hf_2205.xlsx',sheet_name='主次力店202105')
df_hf_202205 =pd.read_excel('~/Downloads/xs_hf_2205.xlsx',sheet_name='主次力店202205')

df1=df_tj.iloc[:,1:]
df2=df_tj.iloc[:,:4]

with pd.ExcelWriter('~/Downloads/11111.xlsx') as writer:
    df1.to_excel(writer,sheet_name='s1')
    df2.to_excel(writer,sheet_name='s1',startrow=4)